In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix, recall_score
from sklearn import model_selection
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.inspection import (partial_dependence, 
                                PartialDependenceDisplay)
sns.set(style='darkgrid', context='talk', palette='Set2')

## Load the Dataset

In [2]:
df_1=pd.read_csv('nbaplayersdraft.csv')
df_1

,id,year,rank,overall_pick,team,player,college,years_active,games,minutes_played,...,3_point_percentage,free_throw_percentage,average_minutes_played,points_per_game,average_total_rebounds,average_assists,win_shares,win_shares_per_48_minutes,box_plus_minus,value_over_replacement
0,1,1989,1,1,SAC,Pervis Ellison,Louisville,11.0,474.0,11593.0,...,0.050,0.689,24.5,9.5,6.7,1.5,21.8,0.090,-0.5,4.4
1,2,1989,2,2,LAC,Danny Ferry,Duke,13.0,917.0,18133.0,...,0.393,0.840,19.8,7.0,2.8,1.3,34.8,0.092,-0.9,4.9
2,3,1989,3,3,SAS,Sean Elliott,Arizona,12.0,742.0,24502.0,...,0.375,0.799,33.0,14.2,4.3,2.6,55.7,0.109,0.2,13.5
3,4,1989,4,4,MIA,Glen Rice,Michigan,15.0,1000.0,34985.0,...,0.400,0.846,35.0,18.3,4.4,2.1,88.7,0.122,0.8,24.9
4,5,1989,5,5,CHH,J.R. Reid,UNC,11.0,672.0,15370.0,...,0.135,0.716,22.9,8.5,5.0,1.0,22.5,0.070,-2.9,-3.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1917,1918,2021,56,56,CHO,Scottie Lewis,Florida,1.0,2.0,7.0,...,NaN,0.500,3.5,0.5,0.0,0.5,0.0,0.164,6.0,0.0
1918,1919,2021,57,57,CHO,Balša Koprivica,Florida State,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919,1920,2021,58,58,NYK,Jericho Sims,Texas,1.0,41.0,555.0,...,NaN,0.414,13.5,2.2,4.1,0.5,1.5,0.128,-1.7,0.0
1920,1921,2021,59,59,BRK,RaiQuan Gray,Florida State,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_final = df_1

In [4]:
df_final = df_final.drop(columns = ['id', 'year', 'rank', 'player',
       'years_active', 'games', 'minutes_played', 'points', 'total_rebounds',
       'assists', 'field_goal_percentage', '3_point_percentage',
       'free_throw_percentage', 'average_minutes_played', 'points_per_game',
       'average_total_rebounds', 'average_assists'])
df_final

,overall_pick,team,college,win_shares,win_shares_per_48_minutes,box_plus_minus,value_over_replacement
0,1,SAC,Louisville,21.8,0.090,-0.5,4.4
1,2,LAC,Duke,34.8,0.092,-0.9,4.9
2,3,SAS,Arizona,55.7,0.109,0.2,13.5
3,4,MIA,Michigan,88.7,0.122,0.8,24.9
4,5,CHH,UNC,22.5,0.070,-2.9,-3.7
...,...,...,...,...,...,...,...
1917,56,CHO,Florida,0.0,0.164,6.0,0.0
1918,57,CHO,Florida State,NaN,NaN,NaN,NaN
1919,58,NYK,Texas,1.5,0.128,-1.7,0.0
1920,59,BRK,Florida State,NaN,NaN,NaN,NaN


In [5]:
df_final = df_final.dropna(axis=0, how='any')
df_final.isna().sum()

overall_pick                 0
team                         0
college                      0
win_shares                   0
win_shares_per_48_minutes    0
box_plus_minus               0
value_over_replacement       0
dtype: int64

In [6]:
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
df_final['overall_pick']= label_encoder.fit_transform(df_final['overall_pick'])

/var/folders/27/gbz_5drj4l13l6x8mnk54j1w0000gn/T/ipykernel_12960/665023397.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['overall_pick']= label_encoder.fit_transform(df_final['overall_pick'])


## Which team perform well ???

In [7]:
df_final

,overall_pick,team,college,win_shares,win_shares_per_48_minutes,box_plus_minus,value_over_replacement
0,0,SAC,Louisville,21.8,0.090,-0.5,4.4
1,1,LAC,Duke,34.8,0.092,-0.9,4.9
2,2,SAS,Arizona,55.7,0.109,0.2,13.5
3,3,MIA,Michigan,88.7,0.122,0.8,24.9
4,4,CHH,UNC,22.5,0.070,-2.9,-3.7
...,...,...,...,...,...,...,...
1914,52,PHI,Western Kentucky,0.8,0.220,1.9,0.2
1915,53,IND,Seton Hall,1.1,0.135,-0.3,0.2
1916,54,OKC,Maryland,1.2,0.048,-4.3,-0.7
1917,55,CHO,Florida,0.0,0.164,6.0,0.0


In [8]:
rating = []
for row in df_final['overall_pick']:
    if row > 30 :    
        rating.append('Not_Peformed_Well')
    else:           
        rating.append('Performed_well')

In [9]:
df_final['rating'] = rating

/var/folders/27/gbz_5drj4l13l6x8mnk54j1w0000gn/T/ipykernel_12960/3626471704.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['rating'] = rating


#### Team

In [10]:
df_team = pd.DataFrame(df_final)

result = df_team.groupby('team')['rating'].apply(lambda x: (x == 'Performed_well').sum() / len(x) * 100)

print(result)

team
ATL     60.000000
BOS     65.573770
BRK     64.285714
CHA     76.470588
CHH     72.727273
CHI     63.934426
CHO     58.333333
CLE     64.444444
DAL     65.853659
DEN     60.000000
DET     58.000000
GSW     58.490566
HOU     60.526316
IND     57.142857
LAC     69.767442
LAL     56.818182
MEM     70.000000
MIA     52.380952
MIL     45.454545
MIN     58.928571
NJN     74.193548
NOH     72.727273
NOK    100.000000
NOP     26.666667
NYK     64.864865
OKC     70.000000
ORL     64.150943
PHI     49.206349
PHO     56.363636
POR     55.319149
SAC     58.333333
SAS     54.545455
SEA     51.612903
TOR     72.413793
UTA     52.830189
VAN     63.636364
WAS     59.259259
WSB     46.153846
Name: rating, dtype: float64


In [11]:
result = result.reset_index()
result

,team,rating
0,ATL,60.000000
1,BOS,65.573770
2,BRK,64.285714
3,CHA,76.470588
4,CHH,72.727273
5,CHI,63.934426
6,CHO,58.333333
7,CLE,64.444444
8,DAL,65.853659
9,DEN,60.000000


In [12]:
result = result.rename(columns = {'rating':'Well_Performed_Percentage'})
result

,team,Well_Performed_Percentage
0,ATL,60.000000
1,BOS,65.573770
2,BRK,64.285714
3,CHA,76.470588
4,CHH,72.727273
5,CHI,63.934426
6,CHO,58.333333
7,CLE,64.444444
8,DAL,65.853659
9,DEN,60.000000


In [13]:
result.to_csv('Team.csv' , index = False)

#### College team

In [14]:
df_col_team = pd.DataFrame(df_final)

result = df_col_team.groupby('college')['rating'].apply(lambda x: (x == 'Performed_well').sum() / len(x) * 100)

print(result)

college
Alabama                     66.666667
Arizona                     63.414634
Arizona State               62.500000
Arkansas                    61.538462
Auburn                      50.000000
                              ...    
Wisconsin                   85.714286
Wright State University    100.000000
Wyoming                     66.666667
Xavier                      45.454545
Yale                         0.000000
Name: rating, Length: 219, dtype: float64


In [15]:
result = result.reset_index()
result

,college,rating
0,Alabama,66.666667
1,Arizona,63.414634
2,Arizona State,62.500000
3,Arkansas,61.538462
4,Auburn,50.000000
...,...,...
214,Wisconsin,85.714286
215,Wright State University,100.000000
216,Wyoming,66.666667
217,Xavier,45.454545


In [16]:
result = result.rename(columns = {'rating':'Well_Performed_Percentage'})
result

,college,Well_Performed_Percentage
0,Alabama,66.666667
1,Arizona,63.414634
2,Arizona State,62.500000
3,Arkansas,61.538462
4,Auburn,50.000000
...,...,...
214,Wisconsin,85.714286
215,Wright State University,100.000000
216,Wyoming,66.666667
217,Xavier,45.454545


In [17]:
result.to_csv('College.csv' , index = False)